### Oppgave 4) Dataanalyse

I denne analysen beregnes tre statistiske mål for temperaturen hvert år i datasettet:

- **Gjennomsnitt (mean):** Gjennomsnittstemperaturen for alle målinger i et gitt år. Dette gir en verdi på det generelle temperaturnivået for hvert år.
- **Median:** Verdien som deler temperaturmålingene i to like store grupper. Halvparten av målingene er lavere enn medianen, og halvparten er høyere. Medianen er mindre påvirket av ekstreme verdier (outliers) enn gjennomsnittet.
- **Standardavvik:** Et mål på hvordan temperaturmålingene i løpet av et år varierer fra gjennomsnittet. Et høyt standardavvik betyr at det har vært store temperaturvariasjoner det året, mens et lavt standardavvik betyr at temperaturene har vært jevnere.

**Nytten av å beregne disse statistiske målene:**

- De gir oss mulighet til å sammenligne år mot år og identifisere trender eller endringer i klimaet over tid.
- Gjennomsnittet viser den generelle utviklingen, mens medianen på den andre siden viser utviklingen når man ser bort ifra enkelttilfeller hvor det har vært ekstreme verdier som trekker veldig opp eller ned på gjennomsnittet.
- Standardavviket hjelper oss å forstå hvor stabile eller varierende temperaturene har vært i de ulike årene.
- Ved å undersøke disse målene kan vi lettere oppdage eventuelle unormale år, identifisere mulige målefeil, og få en bedre forståelse av de klimatiske forholdene i datasettet.

Koden under utfører denne statistiske analysen og gir en oversiktlig tabell med gjennomsnitt, median og standardavvik for hvert år i datasettet.


In [1]:
import sys
sys.path.append('../src')

from temperaturdata import dataanalyse

print(dataanalyse.statistikk_per_aar())
print(dataanalyse.beskriv_data())
# Hvis du vil se renset data (valgfritt):
# print(databehandling.fjern_ekstreme_verdier())


    year  gjennomsnitt  median  stdavvik
0   1950      3.642528  3.5480  9.342967
1   1951      3.329530  3.2480  8.950919
2   1952      2.414126  2.7980  8.662530
3   1953      5.134980  4.8980  8.609882
4   1954      3.345266  3.1480  9.200182
..   ...           ...     ...       ...
70  2020      7.118090  5.8685  6.443153
71  2021      5.749670  5.9185  8.471661
72  2022      6.375646  6.4685  6.778777
73  2023      5.913157  5.1685  7.766205
74  2024      7.337830  6.9685  7.337631

[75 rows x 4 columns]
count    657456.000000
mean          4.313683
std           8.562018
min         -32.202000
25%          -0.952000
50%           4.248000
75%          10.598000
max          31.618500
Name: temperature_2m, dtype: float64


In [1]:
import sys
sys.path.append('../src')

from temperaturdata.dataanalyse import pearson_korrelasjon
pearson_korrelasjon()

Pearson korrelasjon mellom år og temperatur: r = 0.1063, p-verdi = 0.0000


(np.float64(0.10629839453741174), np.float64(0.0))

### Forklaring av korrelasjonsresultat

Korrelasjonsanalysen mellom år og temperatur gir følgende resultat:

- **Pearson korrelasjonskoeffisient (r): 0.1063**
- **p-verdi: 0.0000**

Dette betyr at det finnes en svak, men klar, positiv sammenheng mellom år og temperatur i datasettet. En r-verdi på 0.1063 forteller at temperaturen øker sakte, men sikkert over tid, og at denne økningen er relativt svak sammenlignet med den totale variasjonen i temperaturmålingene.

Den svært lave p-verdien (< 0.05) viser at sammenhengen er statistisk signifikant. Det vil si at det er svært lav sannsynlighet for at denne sammenhengen har oppstått ved en tilfeldighet.

